In [2]:
import pandas as pd
import sqlite3

In [2]:
!pip install pandas==2.0.3
!pip install pyarrow

In [3]:
# restart your kernel here and confirm the pandas version

import pandas as pd
import sqlite3

pd.__version__

In [5]:
bts_columns = [
    "Year",
    "Quarter",
    "Month",
    "DayofMonth",
    "DayOfWeek",
    "FlightDate",
    "Reporting_Airline",
    "DOT_ID_Reporting_Airline",
    "IATA_CODE_Reporting_Airline",
    "Tail_Number",
    "Flight_Number_Reporting_Airline",
    "OriginAirportID",
    "OriginAirportSeqID",
    "OriginCityMarketID",
    "Origin",
    # "OriginCityName",
    # "OriginState",
    "OriginStateFips",
    "OriginStateName",
    "OriginWac",
    "DestAirportID",
    "DestAirportSeqID",
    "DestCityMarketID",
    "Dest",
    # "DestCityName",
    # "DestState",
    "DestStateFips",
    "DestStateName",
    "DestWac",
    "CRSDepTime",
    "DepTime",
    "DepDelay",
    "DepDelayMinutes",
    "DepDel15",
    "DepartureDelayGroups",
    "DepTimeBlk",
    "TaxiOut",
    "WheelsOff",
    "WheelsOn",
    "TaxiIn",
    "CRSArrTime",
    "ArrTime",
    "ArrDelay",
    "ArrDelayMinutes",
    "ArrDel15",
    "ArrivalDelayGroups",
    "ArrTimeBlk",
    "Cancelled",
    "CancellationCode",
    "Diverted",
    "CRSElapsedTime",
    "ActualElapsedTime",
    "AirTime",
    "Flights",
    "Distance",
    "DistanceGroup",
    "CarrierDelay",
    "WeatherDelay",
    "NASDelay",
    "SecurityDelay",
    "LateAircraftDelay",
    "FirstDepTime",
    "TotalAddGTime",
    "LongestAddGTime",
]

bts_query = f"SELECT CASE WHEN SUBSTR(Tail_Number, 1, 1) = 'N' THEN SUBSTR(Tail_Number, 2) ELSE Tail_Number END AS 'N-NUMBER', {', '.join(bts_columns)} FROM on_time"

with sqlite3.connect("bts_on_time.db") as bts_conn:
    df_bts = pd.read_sql_query(bts_query, bts_conn)

In [6]:
with sqlite3.connect("faa_joined.db") as faa_conn:
    df_faa = pd.read_sql_query("SELECT * FROM faa_joined", faa_conn)

In [7]:
df_faa = df_faa.rename(columns={"origin": "faa_registration_origin"})

# df_faa.to_parquet("faa_joined.parquet", engine="pyarrow")

bts_faa_joined = pd.merge(
    df_bts, df_faa, left_on="N-NUMBER", right_on="N-NUMBER", how="left"
)

In [9]:
bts_faa_joined['CRSElapsedTime'] = pd.to_numeric(bts_faa_joined['CRSElapsedTime'], errors='coerce')

In [ ]:
bts_faa_joined.to_parquet("bts_faa_joined.parquet")

In [4]:
df = pd.read_parquet("bts_faa_joined.parquet")

df

FileNotFoundError: [Errno 2] No such file or directory: 'bts_faa_joined.parquet'